In [3]:
import numpy as np
import pandas as pd

class WeatherSimulation:
    """
    Core Weather Simulation using:
    - Poisson Distribution → rain events
    - Binomial Distribution → thunderstorms
    - Normal Distribution → temperature, humidity, rainfall
    - Markov Chain → next day's weather
    """
    
    def __init__(self, days=30):
        self.days = days
        self.weather_data = []
        self.transition_matrix = {
            'Sunny': {'Sunny': 0.6, 'Cloudy': 0.3, 'Rainy': 0.1},
            'Cloudy': {'Sunny': 0.3, 'Cloudy': 0.4, 'Rainy': 0.3},
            'Rainy': {'Sunny': 0.2, 'Cloudy': 0.3, 'Rainy': 0.5}
        }
        
    def markov_transition(self, current_state):
        probabilities = self.transition_matrix[current_state]
        states = list(probabilities.keys())
        probs = list(probabilities.values())
        return np.random.choice(states, p=probs)
    
    def simulate_day(self, current_state):
        rain_events = np.random.poisson(lam=1.5)
        weather_type = None
        temperature = None
        humidity = None
        rainfall = 0
        has_thunderstorm = False
        
        if rain_events == 0:
            if current_state == 'Rainy':
                weather_type = self.markov_transition('Rainy')
            else:
                weather_type = 'Sunny' if np.random.random() < 0.7 else 'Cloudy'
            temperature = np.random.normal(loc=28, scale=5)
            humidity = np.random.normal(loc=60, scale=10)
            rainfall = 0
        elif rain_events == 1:
            weather_type = 'Rainy'
            has_thunderstorm = np.random.binomial(n=1, p=0.2) == 1
            temperature = np.random.normal(loc=22, scale=4)
            humidity = np.random.normal(loc=85, scale=8)
            rainfall = np.random.normal(loc=15, scale=8)
        else:
            weather_type = 'Rainy'
            has_thunderstorm = np.random.binomial(n=1, p=0.4) == 1
            temperature = np.random.normal(loc=20, scale=3)
            humidity = np.random.normal(loc=90, scale=5)
            rainfall = np.random.normal(loc=35, scale=12)
        
        temperature = np.clip(temperature, 15, 40)
        humidity = np.clip(humidity, 30, 100)
        rainfall = max(0, rainfall)
        
        next_state = self.markov_transition(weather_type)
        
        return {
            'weather_type': weather_type,
            'thunder': has_thunderstorm,
            'humidity': humidity,
            'weather_temperature': temperature,
            'rain': rainfall,
            'rain_events': rain_events,
            'next_state': next_state
        }
    
    def run_simulation(self):
        print(f"🌤️  Running Weather Simulation for {self.days} days...\n")
        current_state = 'Sunny'
        for day in range(1, self.days + 1):
            day_weather = self.simulate_day(current_state)
            day_weather['day'] = day
            day_weather['current_state'] = current_state
            self.weather_data.append(day_weather)
            current_state = day_weather['next_state']
        print("✅ Simulation Complete!\n")
        return pd.DataFrame(self.weather_data)

# ===========================
# Ishita’s Part End
# ===========================

# ===========================
# Demo Execution
# ===========================

if __name__ == "__main__":
    simulation = WeatherSimulation(days=30)
    df = simulation.run_simulation()
    
    # Show first 10 days as sample
    print("📋 Sample Weather Data (First 10 days):")
    print(df[['day','weather_type','thunder','humidity','weather_temperature','rain']].head(10).round(2))

🌤️  Running Weather Simulation for 30 days...

✅ Simulation Complete!

📋 Sample Weather Data (First 10 days):
   day weather_type  thunder  humidity  weather_temperature   rain
0    1        Rainy    False     93.34                20.02  42.35
1    2        Sunny    False     63.99                24.86   0.00
2    3        Rainy    False     81.05                25.80   4.48
3    4        Rainy    False     88.96                25.78  47.17
4    5        Rainy    False     91.39                16.51  12.81
5    6        Rainy    False     90.14                16.59  32.18
6    7        Rainy    False     89.35                18.79  33.73
7    8        Rainy     True     85.04                18.41  30.11
8    9        Rainy    False     91.93                21.60  25.57
9   10        Rainy    False     78.01                22.44  33.14
